<hr/>

# Introduction to Data Science
**Tamás Budavári** - budavari@jhu.edu <br/>

- Classification exercises
- Cross-validation

<hr/>

<h1><font color="darkblue">Classification</font></h1>

- Based on a **training set** of labeled points, assign class labels to unknown vectors in the **query set**.  

> **Training set**
><br>
><br>
>$\qquad \displaystyle T = \big\{ (x_i, C_i) \big\}$ 
><br>
><br>
> where $x_i\in \mathbb{R}^d$ are feature sets and $C_i$ are the known class memberships

<nbsp/>

> **Query set**
><br>
><br>
>$\qquad \displaystyle Q = \big\{ x_i \big\}$ 
><br>
><br>
> where $x_i\in \mathbb{R}^d$ consist of the kind of features in $T$

- And again, $x_i$ are not real vectors but **feature sets** of a bunch of scalars in general

### Bayes with Covariance Matrix

- Estimate the full covariance matrix for the classes

>$\displaystyle {\cal{}L}_{\!\boldsymbol{x}}(C_k) =  G(\boldsymbol{x};\mu_k, \Sigma_k)$
><br>
> Handles correlated features well

- Consider binary problem with 2 classes - using Bayes' rule

>$ \displaystyle \frac{P(C_1|x)}{P(C_2|x)} = \frac{\pi_1}{\pi_2}\cdot \frac{{\cal{}L}_{\!\boldsymbol{x}}(C_1)}{{\cal{}L}_{\!\boldsymbol{x}}(C_2)} $

> Taking the negative logarithm, we compare
><br><br>
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma_1^{-1}(x\!-\!\mu_1) + \ln\,\lvert\Sigma_1\lvert $ 
> <br> vs.
><br>
>$\displaystyle (x\!-\!\mu_2)^T\,\Sigma_2^{-1}(x\!-\!\mu_2) + \ln\,\lvert\Sigma_2\lvert $
><br>
><br>
> If the difference is higher/lower than a threshold (based on the priors), we classify $x$ accordingly

- This is called **Quadratic Discriminant Analysis**

### Same Covariance Matrix

- When $\Sigma_1=\Sigma_2=\Sigma$, the quadratic terms cancel from the difference
 
>$\displaystyle (x\!-\!\mu_1)^T\,\Sigma^{-1}(x\!-\!\mu_1) $ 
>$\displaystyle -\ (x\!-\!\mu_2)^T\,\Sigma^{-1}(x\!-\!\mu_2) $

- Hence this is called **Linear Discriminant Analysis**

> Fewer parameters to estimate during the learning process
> <br>
> Good, if we don't have enough data, for example...
> <br>
> Think linear vs quadratic fitting and how you decide between those

### Exercise: QDA 

- Use the provided [training](Class-Train.csv) and [query](Class-Query.csv) sets to perform classification

> **Training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)
> <br>
> **Query** set only has 2 columns of ($x_i$, $y_i$)



>#### Best class?
>$\displaystyle \max_k \big[\ P(C_k|x)\ \big]$
>
>$\displaystyle \max_k \big[\ \pi_k {\cal{}L}_{\!x}(C_k)\ \big]$
>
>$\displaystyle \min_k \big[ -\ln\pi_k - \ln{\cal{}L}_{\!x}(C_k)\ \big]$

> #### Multivariate normal
>$\displaystyle {\cal{}L}_{\!x}(C_k) = \frac{1}{\sqrt{\lvert2\pi\Sigma_k\rvert}} \exp\left(-\frac{1}{2} (x\!-\!\mu_k)^T \Sigma_k^{-1} (x\!-\!\mu_k)\right)$
>
> Hence,
>
>$\displaystyle \min_k \Big[ \frac{1}{2} (x\!-\!\mu_k)^T \Sigma_k^{-1} (x\!-\!\mu_k) + \frac{1}{2}\ln\lvert\Sigma_k\rvert -\ln\pi_k \ \Big]$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class MyQDA(object):
    """ Template for classifier
    """
    def fit(self,X,C):
        self.param = dict()
        # your code here
        return self

    def predict(self,Y):
        Cpred = None
        # your code here
        # use linalg.det(matrix)
        # and linalg.inv(matrix)
        return Cpred

In [3]:
class MyQDA(object):
    """ Simple implementation for illustration purposes
    """       
    def fit(self,X,C):
        self.param = dict()
        for k in np.unique(C):
            members = (C==k)
            prior = members.sum() / C.size
            S = X[members,:] # subset of class 
            mu = S.mean(axis=0)    
            Z = (S-mu).T # centered column vectors
            cov = Z @ Z.T / (Z.shape[1] - 1)
            self.param[k] = (prior, mu, cov)
        return self
            
    def predict(self,Y):
        Cpred = -1 * np.ones(Y.shape[0])
        for i in range(Cpred.size):
            d2min, kbest = 1e99, None
            for k in self.param:
                prior, mu, cov = self.param[k]
                diff = (Y[i,:]-mu).T
                d2 = diff.T @ np.linalg.inv(cov) @ diff / 2
                d2 += np.log(np.linalg.det(cov)) / 2 - np.log(prior) 
                if d2 < d2min: 
                    d2min,kbest = d2,k
            Cpred[i] = kbest
        return Cpred

In [5]:
# reference implementation
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

D = np.loadtxt('Class-Train.csv', delimiter=',')
Q = np.loadtxt('Class-Query.csv', delimiter=',')
X, C = D[:,0:2], D[:,2]

Cpred = MyQDA().fit(X,C).predict(Q)
Cskit =   QDA().fit(X,C).predict(Q)

print ('Number of different estimates:', (Cpred!=Cskit).sum())

Number of different estimates: 0


<h1><font color="darkblue">Cross-Validation</font></h1>

- How to evaluate the quality of estimator?

> $k$-NN method's parameter affects the results

- We saw on the IRIS data that 1-NN was overfitting

> We discussed excluding the point itself

### Partitions of the Training set

- Random complementary subsets 

> Train on a larger subset, test on a small
> <br>
> Multiple rounds to decrease variance

### Leave-One-Out

- For each point, we train on the others and test

> Testing on $n$ points requires $n$ trainings

- Expensive!

### A Relaxed Variant

- $k$-fold cross-validation 

> 1. Create $k$ partitions of equal sizes, e.g., $k=2$ yields two subsets
> 2. Pick a single partition and train on the other $(k\!-\!1)$ 
> 3. Repeat for all $k$ partitions - requires $k$ trainings

- Leave-One-Out is a special case with $k=n$


### Exercise: Cross-Validation

- Evaluate QDA on the [training](files/Class-Train.csv) set using 2-fold cross-validation

> 1. What is the fraction of correct estimates? 
> 2. What is the uncertainty of that fraction?
 
> The **training** set consists of 3 columns of ($x_i$, $y_i$, $C_i$)


In [6]:
Dc = D.copy()
# randomize and split to D1 + D2
np.random.seed(seed=42)
np.random.shuffle(Dc)
split = int(Dc[:,0].size/2)
D1, D2 = Dc[:split,:], Dc[split:,:]

# train on one, estimate on the other
# ... your code here ...

In [7]:
Dc = D.copy()
# randomize and split to D1 + D2
np.random.seed(seed=42)
np.random.shuffle(Dc)
split = Dc.shape[0] // 2
D1, D2 = Dc[:split,:], Dc[split:,:]
# train on one estimate or the other
for i,(T,Q) in enumerate([(D1,D2),(D2,D1)]):
    X, C = T[:,0:2], T[:,2]
    Cpred, Ctrue = MyQDA().fit(X,C).predict(Q[:,:2]), Q[:,2]
    print ("Case #%d - Number of mislabeled points out of a total %3d points : %2d" \
        % (i, Q.shape[0],(Ctrue!=Cpred).sum()))

Case #0 - Number of mislabeled points out of a total 157 points : 19
Case #1 - Number of mislabeled points out of a total 156 points : 20


### Done already?

- Visualize the results in the 2D features space
- Make these simple codes run faster 


### 3-fold CV - quick hack

In [8]:
Dc = D.copy()
print(Dc)
# randomize and split to D1 + D2
np.random.seed(seed=42)
np.random.shuffle(Dc)
split = int(Dc[:,0].size/3)
split2 = 2*split
D1, D2, D3 = Dc[:split,:], Dc[split:split2,:], Dc[split2:]

# train on one, estimate on the other
for T,Q in [ (np.vstack([D1,D2]),D3), (np.vstack([D2,D3]),D1), (np.vstack([D3,D1]),D2)]:
    Cpred = QDA().fit(T[:,:2],T[:,2]).predict(Q[:,:2])
    Ctrue = Q[:,2]
    print ((Cpred!=Ctrue).sum(), T.shape)

[[ -5.072856   -12.93877095   0.        ]
 [ -5.46445715 -12.73849219   0.        ]
 [-10.88780154 -20.1482205    0.        ]
 [ -9.03783938 -16.53543305   0.        ]
 [ -6.22186344 -11.02917434   0.        ]
 [-11.49811837 -14.01356862   0.        ]
 [ -6.28537085 -11.03069691   0.        ]
 [ -6.78279726 -10.7631925    0.        ]
 [ -8.23295389 -13.99594848   0.        ]
 [ -6.2276255  -10.50494013   0.        ]
 [ -7.33747828 -14.86012719   0.        ]
 [ -4.33245413  -9.53281179   0.        ]
 [-12.23123917 -18.82298876   0.        ]
 [ -4.6560809  -13.19907263   0.        ]
 [ -7.59732258 -13.34147199   0.        ]
 [ -5.172616   -11.14532048   0.        ]
 [ -8.05464277 -13.35119947   0.        ]
 [ -6.72881426 -15.51670133   0.        ]
 [ -4.82783373 -11.00577341   0.        ]
 [ -6.77325222 -11.74555133   0.        ]
 [ -4.811992   -11.00788116   0.        ]
 [-11.24391376 -21.49968249   0.        ]
 [ -6.84242298 -13.27265215   0.        ]
 [-10.61457444 -16.03819023   0.  

### Unhomework

- Implement LDA and compare to sklearn

>1. Write code without using sklearn 
>2. Apply to [training](Class-Train.csv) and [query](Class-Query.csv) sets 
>3. Compare your results to sklearn's 

- Perform 10-fold cross-validation of *MyQDA* on [this](Class-Train.csv) file

>1. Write code without using `sklearn` 
>2. Calculate average number of good classifications 
>3. Compare to sklearn 

In [9]:
from sklearn.model_selection import cross_val_score
clf = QDA()
cross_val_score(clf, X,C, cv=10)

array([0.8125    , 0.875     , 0.75      , 0.875     , 0.8125    ,
       0.75      , 1.        , 1.        , 1.        , 0.93333333])

### What does this mean?

In [10]:
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=False) 

for k, (train, test) in enumerate(k_fold.split(X)):
    clf.fit(X[train],C[train])
    Cpred = clf.predict(X[test])
    print (k, ':\t', (C[test]==Cpred).sum() / float(test.size),
        '  =  ', clf.score(X[test],C[test]) )

0 :	 0.8125   =   0.8125
1 :	 0.875   =   0.875
2 :	 0.75   =   0.75
3 :	 0.875   =   0.875
4 :	 0.8125   =   0.8125
5 :	 0.75   =   0.75
6 :	 1.0   =   1.0
7 :	 1.0   =   1.0
8 :	 0.9333333333333333   =   0.9333333333333333
9 :	 1.0   =   1.0
